In [1]:
!/opt/bin/nvidia-smi

系统找不到指定的路径。


In [2]:
pip install huggingface-hub==0.0.12
pip install ipython==5.5.0
pip install ipykernel==4.10
pip install simpletransformers

SyntaxError: invalid syntax (<ipython-input-2-b173e515fca1>, line 1)

In [1]:
# 导入依赖包
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import accuracy_score

#数据读取
total_df = pd.read_csv('data_translated_with_mark.csv')

total_df.columns = ['uig', 'label']
total_df['label'] = total_df['label'].astype(int)
#total_df = total_df.reindex(columns=['text', 'label'])
#total_df['label'] = total_df['label'].map({'like': 0, 'disgust': 1, 'sadness':1, 'happiness': 0, 'anger': 1, 'surprise': 0, 'fear': 1})

train_df, test_df = train_test_split(total_df, test_size=0.2, random_state=42, shuffle=True)

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df = train_df.rename(columns={'label':'labels'})
test_df = test_df.rename(columns={'label':'labels'})

train_df.head(10)

,uig,labels
0,جۇڭگونىڭ گۈللىنىشى، مۇقىملىقى نېپال ئۈچۈنمۇ پۇ...,1
1,شۇ كۈنى كۇبا مەملىكەتلىك خەلق ھۆكۈمىتى ۋەكىللە...,0
2,بۈگۈنكى كۈندە، چېن لاۋچى كەنتىنىڭ قىياپىتى يېڭ...,1
3,كەنت ئاھالىلىرىنىڭ رەڭدار تۇرمۇشى ئاياللارنىڭ ...,1
4,«چىرايلىق كىيىملەرنى كىيىپ ئۆيدىن چىقىپ ئىككى ...,0
5,ئەركىن سودا رايونى قۇرۇلغاندىن بۇيان، ئىشلار...,1
6,ياش دوستلار، يولداشلار!,1
7,باش تېمىلىق ناخشا «گۈزەل يۇرت - ماكان»دا يۇرت ...,0
8,چۈشتىن بۇرۇن سائەت 10دىن 30 مىنۇت ئۆتكەندە، شى...,0
9,6-ئاينىڭ 13-كۈنى (پەيشەنبە)دىن باشلاپ، ھاۋا يا...,0


In [2]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
from transformers import AutoModel
model_name = "xlm-roberta-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [3]:
import sys
new_uig_tokens=[]
with open('uig.txt','r', encoding = 'utf-8') as f:
    for line in f:
        token = str(line.strip('\n').split(','))[2:-2]
        if token.find('##'):
            #token = token.replace('##', '▁')
            new_uig_tokens.append(token)
new_uig_tokens = new_uig_tokens[0:1027]
print(new_uig_tokens)

['بىر', 'بىلەن', 'ۋە', 'ئۇ', 'دەپ', 'مەن', 'بولۇپ', '–', 'قىلىپ', 'بولغان', 'ياخشى', 'شۇ', 'ئۈچۈن', 'يەنە', 'بولسا', 'ئۇنىڭ', 'بولىدۇ', 'gt', 'ياكى', 'قىلىش', 'ئىدى', 'يوق', 'ئۇيغۇر', 'ھەر', 'ئۆز', 'ئەمەس', 'كۆپ', 'ئىش', 'قىلغان', 'بىز', 'ئەڭ', 'قانداق', 'باشقا', 'ئىككى', 'كېرەك']


35

In [4]:
print("[ BEFORE ] tokenizer vocab size:", len(tokenizer)) 
added_tokens = tokenizer.add_tokens(new_uig_tokens)

print("[ AFTER ] tokenizer vocab size:", len(tokenizer)) 
print('added_tokens:',added_tokens)


[ BEFORE ] tokenizer vocab size: 250002
[ AFTER ] tokenizer vocab size: 250027

added_tokens: 25



In [5]:
model = ClassificationModel('xlmroberta', 'xlm-roberta-base', num_labels=3, args={
      'sliding_window': False,
      'save_steps': False, 
      'reprocess_input_data': True, 
      'overwrite_output_dir': True, 
      'logging_steps': 50, 
      'train_batch_size': 15, 
      'gradient_accumulation_steps': 1, 
      'stride': 0.8, 'max_seq_length': 185, 
      'num_train_epochs': 14, 
      'fp16': False, 
      'reprocess_input_data': True,
      'weight_decay': 0.0005,
      'optimizer': "AdamW",
      'evaluate_during_training':True,
      'special_tokens_list':new_uig_tokens,
      'warmup_ratio': 0.06, 
      'learning_rate': 2e-6})

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weig

In [6]:
print('get model')
model.train_model(train_df, eval_df=test_df)
test_predict = model.predict([row['uig'] for _, row in test_df.iterrows()])
print(accuracy_score(test_df['labels'], test_predict[0]))

get model


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\simpletransformers\classification\classification_model.py:449: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\simpletransformers\classification\classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\simpletransformers\classification\classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\simpletransformers\classification\classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


...

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\simpletransformers\classification\classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\simpletransformers\classification\classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\simpletransformers\classification\classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\simpletransformers\classification\classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



0.7470956210902592


In [7]:
models_list=[]
for i in range(1,15):
    models_name = r"./outputs/checkpoint-"+str(299*i)+"-epoch-"+str(i)
    models_list.append(models_name)
print(models_list)

counter = 1
df = pd.DataFrame()

for model_addr in models_list:
    info = "Senti-eXLM-"+str(299*counter)+"-epoch-"+str(counter)
    #info = "XLM-R-"+str(2000*counter)+"checkpoint"
    print(info)
    model = ClassificationModel('xlmroberta', model_addr, num_labels=3, args={'sliding_window': False, 'reprocess_input_data': True, 'overwrite_output_dir': False, 'logging_steps': 50, 'train_batch_size': 32, 'gradient_accumulation_steps': 1, 'stride': 0.8, 'max_seq_length': 185, 'num_train_epochs': 50, 'fp16': False, 'reprocess_input_data': True, 'learning_rate': 1e-5})
    test_predict = model.predict([row['uig'] for _, row in test_df.iterrows()])
    print(accuracy_score(test_df['labels'], test_predict[0]))
    
    res_accuracy_score = accuracy_score(test_df['labels'].tolist(), test_predict[0])
    res_precision_score = precision_score(test_df['labels'].tolist(), test_predict[0], average='weighted')
    res_recall_score = recall_score(test_df['labels'].tolist(), test_predict[0], average='weighted')
    res_f1_score = f1_score(test_df['labels'].tolist(), test_predict[0], average='weighted')

    df = df.append([[20210712, info, res_accuracy_score, res_precision_score, res_recall_score, res_f1_score]])
    counter = counter+1
    
df.columns=['time', 'model', 'accuracy_score', 'precision_score', 'recall_score' ,'f1_score']
df.to_csv('XLM-info03-epochs.csv', index=False)

['./outputs/checkpoint-299-epoch-1', './outputs/checkpoint-598-epoch-2', './outputs/checkpoint-897-epoch-3', './outputs/checkpoint-1196-epoch-4', './outputs/checkpoint-1495-epoch-5', './outputs/checkpoint-1794-epoch-6', './outputs/checkpoint-2093-epoch-7', './outputs/checkpoint-2392-epoch-8', './outputs/checkpoint-2691-epoch-9', './outputs/checkpoint-2990-epoch-10', './outputs/checkpoint-3289-epoch-11', './outputs/checkpoint-3588-epoch-12', './outputs/checkpoint-3887-epoch-13', './outputs/checkpoint-4186-epoch-14']
Senti-eXLM-299-epoch-1


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\simpletransformers\classification\classification_model.py:346: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  f"use_multiprocessing automatically disabled as {model_type}"



0.6729222520107239
Senti-eXLM-598-epoch-2


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\simpletransformers\classification\classification_model.py:346: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  f"use_multiprocessing automatically disabled as {model_type}"



0.6934763181411975
Senti-eXLM-897-epoch-3


...


0.7515638963360143
Senti-eXLM-3887-epoch-13


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\simpletransformers\classification\classification_model.py:346: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  f"use_multiprocessing automatically disabled as {model_type}"



0.743521000893655
Senti-eXLM-4186-epoch-14


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\simpletransformers\classification\classification_model.py:346: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  f"use_multiprocessing automatically disabled as {model_type}"



0.7506702412868632


In [8]:
models_list=['./outputs/best_model']

print(models_list)

counter = 1
df = pd.DataFrame()

for model_addr in models_list:
    info = "Senti-eXLM-"+str(299*counter)+"-epoch-"+str(counter)
    #info = "XLM-R-"+str(2000*counter)+"checkpoint"
    print(info)
    model = ClassificationModel('xlmroberta', model_addr, num_labels=3, args={'sliding_window': False, 'reprocess_input_data': True, 'overwrite_output_dir': False, 'logging_steps': 50, 'train_batch_size': 32, 'gradient_accumulation_steps': 1, 'stride': 0.8, 'max_seq_length': 185, 'num_train_epochs': 50, 'fp16': False, 'reprocess_input_data': True, 'learning_rate': 1e-5})
    test_predict = model.predict([row['uig'] for _, row in test_df.iterrows()])
    print(accuracy_score(test_df['labels'], test_predict[0]))
    
    res_accuracy_score = accuracy_score(test_df['labels'].tolist(), test_predict[0])
    res_precision_score = precision_score(test_df['labels'].tolist(), test_predict[0], average='weighted')
    res_recall_score = recall_score(test_df['labels'].tolist(), test_predict[0], average='weighted')
    res_f1_score = f1_score(test_df['labels'].tolist(), test_predict[0], average='weighted')

    df = df.append([[20210712, info, res_accuracy_score, res_precision_score, res_recall_score, res_f1_score]])
    counter = counter+1
    
df.columns=['time', 'model', 'accuracy_score', 'precision_score', 'recall_score' ,'f1_score']
df.to_csv('XLM-info03-epochs.csv', index=False)

['./outputs/best_model']
Senti-eXLM-299-epoch-1


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\simpletransformers\classification\classification_model.py:346: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  f"use_multiprocessing automatically disabled as {model_type}"



0.7497765862377123
